In [1]:
import geopandas as gpd
import pandas as pd
import os
import networkx as nx
import osmnx as ox
import matplotlib.pyplot as plt
import json
import geocoder
import folium
import fiona
import geemap
import xml
from pyproj import CRS
from shapely.geometry.polygon import LinearRing, Polygon, Point
from shapely.geometry import MultiPolygon, Point, MultiPoint, MultiLineString
%matplotlib inline

In [65]:
%pwd

'D:\\ALL_data_science\\Freelancing\\UPwork\\Super Software'

# using KML file

In [102]:
# Enable KML driver
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

In [4]:
# Read file from KML
#fp = "Sample (4).kml"
df = gpd.read_file("Sample (4).kml", driver='KML')

In [6]:
df.sample(3)

,Name,Description,geometry
107,,,"MULTIPOLYGON (((-93.72488 38.78091, -93.72488 ..."
204,,,"MULTIPOLYGON (((-100.39546 31.48261, -100.3958..."
799,,,"MULTIPOLYGON (((-97.53391 32.99568, -97.53440 ..."


In [96]:
# parsing the kmz file
from zipfile import ZipFile

filename = 'national_frs.kmz'

kmz = ZipFile(filename, 'r')
#kml = kmz.open('doc.kml', 'r')
kmz.filelist

[<ZipInfo filename='NATIONAL_FRS.KML' compress_type=deflate filemode='-rw-r--r--' file_size=88436523 compress_size=4493767>]

In [100]:
kml = kmz.extract('NATIONAL_FRS.KML')               

In [103]:
# Read file from KML
#fp = "Sample (4).kml"
df2 = gpd.read_file("NATIONAL_FRS.KML", driver='KML')

df2.explore(
)

# Next, we need to convert this GeoDataFrame to “Azimuthal Equidistant” -projection that has useful properties because all points on the map in that projection are at proportionately correct distances from the center point (defined with parameters lat_0 and lon_0), and all points on the map are at the correct direction from the center point.

# To conduct the transformation, we are going to utilize again pyproj library which is also good at dealing with “special” projections such as the one demonstrated here.

In [7]:
# Reproject to aeqd projection using Proj4-string
#df = df.to_crs('+proj=aeqd')
df = df.to_crs('ESRI:102009')

In [8]:
df.crs

<Derived Projected CRS: ESRI:102009>
Name: North_America_Lambert_Conformal_Conic
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. United States (USA) - Alabama; Alaska (mainland); Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming.
- bounds: (-172.54, 23.81, -47.74, 86.46)

In [9]:
# using a little buffer to avoid self intersected polygons from throwing errors while converting

In [10]:
df_exploded = df.explode(column='geometry', ignore_index=True)

In [11]:
df_exploded.dropna(inplace=True
                           )

In [12]:
df_exploded

,Name,Description,geometry
0,,,"POLYGON ((1742984.758 295931.959, 1742966.100 ..."
1,,,"POLYGON ((1329603.431 -1265983.091, 1329581.61..."
2,,,"POLYGON ((781844.545 -362106.124, 781801.555 -..."
3,,,"POLYGON ((783016.190 -359067.229, 783016.186 -..."
4,,,"POLYGON ((53971.489 -1073750.435, 53971.342 -1..."
...,...,...,...
1003,,,"POLYGON ((18982.574 -1096521.690, 18982.325 -1..."
1004,,,"POLYGON ((1435231.796 -1097120.960, 1435259.66..."
1005,,,"POLYGON ((-82867.184 -715330.007, -82877.167 -..."
1006,,,"POLYGON ((215041.811 523213.038, 215041.502 52..."


In [13]:
#df_exploded['geometry'] = df_exploded.buffer(0.01)

In [14]:
df_exploded.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1007 entries, 0 to 1007
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Name         1007 non-null   object  
 1   Description  1007 non-null   object  
 2   geometry     1007 non-null   geometry
dtypes: geometry(1), object(2)
memory usage: 31.5+ KB


# extracting polygons from multipolygons
lines = []
for value in df["geometry"]:
    line = value.boundary
    lines.append(line)
          
df["Linestrings"] = lines

In [15]:
df_exploded.sample(3)

,Name,Description,geometry
288,,,"POLYGON ((-1262942.836 240489.868, -1262961.96..."
373,,,"POLYGON ((510319.721 94776.024, 510311.144 947..."
753,,,"POLYGON ((1420130.466 -1075104.173, 1420124.37..."


In [16]:
df_exploded.dropna(inplace=True)

In [17]:
df_exploded.isna().sum()

Name           0
Description    0
geometry       0
dtype: int64

In [18]:
from shapely.geometry import LineString, LinearRing

In [19]:
df_exploded = df_exploded.reset_index()

In [20]:
df_exploded.rename(columns={'index':'id'}, inplace=True)

# Getting the polygong segments lengths

In [21]:
# Facts about polygon outer vertices
# - first vertex is the same as the last
# - to get segments, ignore zero-th point (use it as from_point in next row)
# create basic lists for creation of new dataframe
indx = []  # for A1, A3
sequ = []  # for seg order
pxy0 = []  # from-point
pxy1 = []  # to-point
for ix,geom in zip(df_exploded.id, df_exploded.geometry):
    num_pts = len(geom.exterior.xy[0])
    #print(ix, "Num points:", num_pts)
    old_xy = []
    for inx, (x,y) in enumerate(zip(geom.exterior.xy[0],geom.exterior.xy[1])):
        if (inx==0):
            # first vertex is the same as the last
            pass
        else:
            indx.append(ix)
            sequ.append(inx)
            pxy0.append(Point(old_xy))
            pxy1.append(Point(x,y))
        old_xy = (x,y)

# Create new geodataframe 
pgon_segs  = gpd.GeoDataFrame({"poly_id": indx,
                 "vertex_id": sequ,
                 "fr_point": pxy0,
                 "to_point": pxy1}, geometry="to_point")
# Compute segment lengths
# Note: seg length is Euclidean distance, ***not geographic***
pgon_segs["seg_length"] = pgon_segs.apply(lambda row: row.fr_point.distance(row.to_point), axis=1)


In [22]:
pgon_segs

,poly_id,vertex_id,fr_point,to_point,seg_length
0,0,1,POINT (1742984.757588502 295931.9587225499),POINT (1742966.100 295954.424),29.202663
1,0,2,POINT (1742966.099778431 295954.4238428031),POINT (1742955.160 295945.126),14.357377
2,0,3,POINT (1742955.159838006 295945.1258868397),POINT (1742973.818 295922.661),29.202735
3,0,4,POINT (1742973.817770591 295922.6607746293),POINT (1742984.758 295931.959),14.357279
4,1,1,POINT (1329603.430602161 -1265983.090926219),POINT (1329581.616 -1265986.603),22.095633
...,...,...,...,...,...
16463,1007,1,POINT (1733720.371282645 292019.1236909979),POINT (1733709.029 292046.159),29.317971
16464,1007,2,POINT (1733709.028539483 292046.1585868575),POINT (1733702.498 292043.372),7.100506
16465,1007,3,POINT (1733702.497661906 292043.3720438288),POINT (1733713.840 292016.337),29.317879
16466,1007,4,POINT (1733713.840429029 292016.3372575118),POINT (1733718.739 292018.427),5.325319


In [23]:
pgon_segs['line'] = pgon_segs.apply(lambda row: LineString([row['fr_point'], row['to_point']]), axis=1)

C:\Users\Smile2DeaTH\anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [24]:
pgon_segs.rename(columns={'line':'geometry'}, inplace=True)

In [25]:
pgon_segs = pgon_segs.set_geometry('geometry')

In [26]:
pgon_segs = pgon_segs.set_crs("ESRI:102009")

In [27]:
pgon_segs

,poly_id,vertex_id,fr_point,to_point,seg_length,geometry
0,0,1,POINT (1742984.757588502 295931.9587225499),POINT (1742966.100 295954.424),29.202663,"LINESTRING (1742984.758 295931.959, 1742966.10..."
1,0,2,POINT (1742966.099778431 295954.4238428031),POINT (1742955.160 295945.126),14.357377,"LINESTRING (1742966.100 295954.424, 1742955.16..."
2,0,3,POINT (1742955.159838006 295945.1258868397),POINT (1742973.818 295922.661),29.202735,"LINESTRING (1742955.160 295945.126, 1742973.81..."
3,0,4,POINT (1742973.817770591 295922.6607746293),POINT (1742984.758 295931.959),14.357279,"LINESTRING (1742973.818 295922.661, 1742984.75..."
4,1,1,POINT (1329603.430602161 -1265983.090926219),POINT (1329581.616 -1265986.603),22.095633,"LINESTRING (1329603.431 -1265983.091, 1329581...."
...,...,...,...,...,...,...
16463,1007,1,POINT (1733720.371282645 292019.1236909979),POINT (1733709.029 292046.159),29.317971,"LINESTRING (1733720.371 292019.124, 1733709.02..."
16464,1007,2,POINT (1733709.028539483 292046.1585868575),POINT (1733702.498 292043.372),7.100506,"LINESTRING (1733709.029 292046.159, 1733702.49..."
16465,1007,3,POINT (1733702.497661906 292043.3720438288),POINT (1733713.840 292016.337),29.317879,"LINESTRING (1733702.498 292043.372, 1733713.84..."
16466,1007,4,POINT (1733713.840429029 292016.3372575118),POINT (1733718.739 292018.427),5.325319,"LINESTRING (1733713.840 292016.337, 1733718.73..."


In [28]:
nodes_df = pgon_segs[['poly_id','vertex_id','fr_point','to_point']]

In [29]:
nodes_gdf = gpd.GeoDataFrame(nodes_df, geometry='fr_point', crs=pgon_segs.crs)

In [46]:
nodes_gdf.sample(3)

,poly_id,vertex_id,fr_point,to_point
2926,168,262,POINT (1421271.074 -1075291.531),POINT (1421270.897 -1075291.265)
1820,107,100,POINT (1445573.074 -1140659.083),POINT (1445573.008 -1140658.951)
11940,784,5,POINT (-1248970.955 186794.575),POINT (-1248961.843 186842.833)


In [31]:
pgon_segs.drop(columns=['fr_point','to_point'], inplace=True)

In [32]:
pgon_segs['seg_length'] = round(pgon_segs['seg_length'], 1) 

In [33]:
pgon_segs.head(3)

,poly_id,vertex_id,seg_length,geometry
0,0,1,29.2,"LINESTRING (1742984.758 295931.959, 1742966.10..."
1,0,2,14.4,"LINESTRING (1742966.100 295954.424, 1742955.16..."
2,0,3,29.2,"LINESTRING (1742955.160 295945.126, 1742973.81..."


m = pgon_segs.explore(tooltip='seg_length'

)
nodes_gdf['fr_point'].explore(m=m,
                  color='red'
                 )

nodes_gdf['to_point'].explore(m=m,
                  color='red'
                 )

df_exploded.explore(m=m, color='green'
                   )

folium.TileLayer('Cartodb dark_matter', control=True).add_to(m)  # use folium to add alternative tiles
folium.LayerControl().add_to(m)  # use folium to add layer control
m


# importing Qgis shp files

In [42]:
qgis_lines = gpd.read_file(r"qgis_exports/geom_info.shp")

In [24]:
qgis_lines['length'] = qgis_lines['length'].round(1)

In [25]:
qgis_corners = gpd.read_file(r"qgis_exports/corner_nodes.shp")

In [26]:
qgis_simp_poly = gpd.read_file(r"qgis_exports/simplified_polygons.shp")

In [239]:
qgis_lines = qgis_lines.iloc[0:199]
sample_corners = qgis_corners.iloc[0:199]
sample_polygons = qgis_simp_poly.iloc[0:199]

In [20]:
qgis_lines.to_file('sample_200/_lines.shp',crs=qgis_lines.crs)
qgis_corners.to_file('sample_200/_corners.shp',crs=qgis_corners.crs)
qgis_simp_poly.to_file('sample_200/_polygons.shp',crs=qgis_simp_poly.crs)

In [ ]:
# Picking one city address for testing in this case "Chicago"

In [111]:
qgis_simp_poly.query("mdCity == 'Chicago'")['ParcelNu_1']

36     20272090220000
58     13283300040000
90     16234160140000
118    16023110020000
167    20072240370000
241    13141250330000
282    19133210100000
294    16122020491001
346    13122270120000
383    16041040290000
419    14074040160000
426    20074170290000
449    20272030140000
451    16263080320000
470    20262070321003
535    20282060300000
563    16111320130000
580    13341220240000
591    19244160070000
600        1131409017
606    20171090400000
612    20174050210000
626    20182220420000
628    16261020370000
656     9163040160000
732    16251080030000
748    14203040180000
780    16243140160000
856    16224070280000
858    13321130080000
867    20213260080000
868    20074130320000
903    13012020150000
907    13123100200000
986    13293020020000
Name: ParcelNu_1, dtype: object

In [47]:
qgis_lines['mdAddress'].value_counts()  

26883 Park Terrace Ln    224
13229 Alta Vista Way      56
26486 Olivewood           37
7495 Mill Pond Cir        35
445 Marsac Ave            32
                        ... 
8412 W Crain St            4
6507 Cottage Ln            4
308 Malcolm X Blvd         4
3340 RIVES EATON RD        3
40010 River Rd             3
Name: mdAddress, Length: 955, dtype: int64

# addresses free form
- 26883 Park Terrace Ln, Mission Viejo, CA, 92692-6116
- 13229 Alta Vista Way, Sylmar, CA, 91342-3464
- 26486 Olivewood, Lake Forest, CA, 92630-6723
- 7495 Mill Pond Cir, Naples, FL, 34109-1708
- 445 Marsac Ave, Park City, UT, 84060-5122

# using Mellisa API  with requests lib

- seems that requests needs more modifications to be used

In [2]:
import requests
import json

In [3]:
url = "https://property.melissadata.net/v4/WEB/LookupProperty?id=eSDChydmJdiUYj-c8V5aqV**nSAcwXpxhQ0PC2lXxuDAZ-**&t=test&cols=GrpAll&format=JSON&ff=8700 sunlit cove dr. St. Petersburg, FL"

payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)



requests_data = response.text

requests_data = json.loads(requests_data)
final_request_data = pd.json_normalize(requests_data['Records'])



In [ ]:
final_request_data

,Results,AssociatedParcels,Parcel.FIPSCode,Parcel.County,Parcel.UnformattedAPN,Parcel.FormattedAPN,Parcel.AlternateAPN,Parcel.APNYearChange,Parcel.PreviousAPN,Parcel.AccountNumber,...,YardGardenInfo.ArenaFlag,YardGardenInfo.WaterFeatureFlag,YardGardenInfo.PondFlag,YardGardenInfo.BoatLiftFlag,EstimatedValue.EstimatedValue,EstimatedValue.EstimatedMinValue,EstimatedValue.EstimatedMaxValue,EstimatedValue.ConfidenceScore,EstimatedValue.ValuationDate,Shape.WellKnownText
0,"YS02,YS07,YC01,GS05",[],12103,Pinellas,19-30-17-86634-007-0080,193017866340070080,,,,R210766,...,,,,,347000,291480,402520,84,20211222,POLYGON ((-82.634174745999985 27.8520448490000...


In [35]:
final_request_data.to_csv('address.csv')

In [36]:
%pwd

'D:\\ALL_data_science\\Freelancing\\UPwork\\Super Software'

## Tested mellisa api with rest link on POSTMAN
## and here invetigating the downloaded result

In [ ]:
from shapely.geometry import Polygon
import shapely

data = pd.read_json('Melissa API\\now', orient='columns')

data_2 = pd.json_normalize(data['Records'])

In [259]:
#data_2.to_csv("Melissa API\\updated_request_na.csv", na_rep='NA')

# Create ‘out.zip’ containing ‘out.csv’
compression_opts = dict(method='zip',
                        archive_name='out.csv')  
df.to_csv('out.zip', index=False,
          compression=compression_opts) 

In [392]:
gdf_melissa = gpd.GeoDataFrame(final_request_data)

In [393]:
gdf_melissa.rename(columns={'Shape.WellKnownText':'geometry'},inplace=True)

In [394]:
geometry = gdf_melissa['geometry'].map(shapely.wkt.loads) # mapping the geometry
gdf_melissa = gpd.GeoDataFrame(gdf_melissa, crs=('WGS84'), geometry=geometry)

In [395]:
gdf_melissa.reset_index(inplace=True)
gdf_melissa.rename(columns={'index':'id'}, inplace=True)

In [396]:
melissa_exploded = gdf_melissa.explode(column='geometry', ignore_index=True)

In [397]:
melissa_exploded.geometry.simplify(1)

0    POLYGON ((-117.65013 33.57209, -117.65043 33.5...
dtype: geometry

In [409]:
x, y = melissa_exploded.geometry[0].exterior.coords.xy

In [414]:
type(x)

array.array

In [398]:
# Facts about polygon outer vertices
# - first vertex is the same as the last
# - to get segments, ignore zero-th point (use it as from_point in next row)
# create basic lists for creation of new dataframe
indx = []  # for A1, A3
sequ = []  # for seg order
pxy0 = []  # from-point
pxy1 = []  # to-point
for ix,geom in zip(melissa_exploded.index, melissa_exploded.geometry):
    num_pts = len(geom.exterior.xy[0])
    #print(ix, "Num points:", num_pts)
    old_xy = []
    for inx, (x,y) in enumerate(zip(geom.exterior.xy[0],geom.exterior.xy[1])):
        if (inx==0):
            # first vertex is the same as the last
            pass
        else:
            indx.append(ix)
            sequ.append(inx)
            pxy0.append(Point(old_xy))
            pxy1.append(Point(x,y))
        old_xy = (x,y)

# Create new geodataframe 
melissa_segs  = gpd.GeoDataFrame({"poly_id": indx,
                 "vertex_id": sequ,
                 "fr_point": pxy0,
                 "to_point": pxy1}, geometry="to_point", crs='ESRI:102009')
# Compute segment lengths
# Note: seg length is Euclidean distance, ***not geographic***
melissa_segs["seg_length"] = melissa_segs.apply(lambda row: row.fr_point.distance(row.to_point), axis=1)


In [399]:
melissa_segs.geometry = melissa_segs.fr_point

In [400]:
melissa_segs.iloc[0]

poly_id                                              0
vertex_id                                            1
fr_point      POINT (-117.650128071 33.57209265100005)
to_point      POINT (-117.650128071 33.57209265100005)
seg_length                                    0.000358
Name: 0, dtype: object

In [403]:
from shapely.geometry import Point, mapping, shape

In [405]:
print mapping(melissa_segs['fr_point'][0])

SyntaxError: invalid syntax (Temp/ipykernel_19392/4190484504.py, line 1)

gdf_melissa.explore(
)

-----------------------------

# working with the shp files from ArcGis

In [240]:
with open('ArcGis files\\nuke.dbf.xml') as f:
    sml = f.read()

In [ ]:
### solution from medium

In [257]:
import xml.etree.ElementTree as et 

xtree = et.parse("ArcGis files\\nuke.dbf.xml")
xroot = xtree.getroot()

In [ ]:
def parse_XML(xml_file, df_cols): 
    """Parse the input XML file and store the result in a pandas 
    DataFrame with the given columns. 
    
    The first element of df_cols is supposed to be the identifier 
    variable, which is an attribute of each node element in the 
    XML data; other features will be parsed from the text content 
    of each sub-element. 
    """
    
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()
    rows = []
    
    for node in xroot: 
        res = []
        res.append(node.attrib.get(df_cols[0]))
        for el in df_cols[1:]: 
            if node is not None and node.find(el) is not None:
                res.append(node.find(el).text)
            else: 
                res.append(None)
        rows.append({df_cols[i]: res[i] 
                     for i, _ in enumerate(df_cols)})
    
    out_df = pd.DataFrame(rows, columns=df_cols)
        
    return out_df

--------------------------------

In [ ]:
from shapely.geometry import Point,LineString, Polygon
from shapely.geometry import MultiPoint, MultiLineString, MultiPolygon, box
from folium import Map, Marker, GeoJson, LayerControl
from descartes import PolygonPatch

# converting coooredinate points to lat/lon

In [9]:
# converting coooredinate points to lat/lon
def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

In [14]:
xy_to_lonlat(1353844.24, 1051503.78)

C:\Users\SMILE2~1\AppData\Local\Temp/ipykernel_4880/3998854258.py:4: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)


(22.754963395595265, 9.426418934470277)

In [18]:
point_= (22.754963395595265, 9.426418934470277)
tanks = ox.geometries.geometries_from_point(center_point=point_, tags = {'man_made':'storage_tank'}, dist=1000)

# Using the donwlaoded files from EPA

In [132]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import dask.array as da
import dask.bag as db

In [137]:
alrernative_name = pd.read_csv("EPA API\\downloads\\national_combined\\NATIONAL_ALTERNATIVE_NAME_FILE.CSV")

In [138]:
alrernative_name.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5527820 entries, 0 to 5527819
Data columns (total 5 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   REGISTRY_ID            int64 
 1   PGM_SYS_ACRNM          object
 2   PGM_SYS_ID             object
 3   ALTERNATIVE_NAME       object
 4   ALTERNATIVE_NAME_TYPE  object
dtypes: int64(1), object(4)
memory usage: 210.9+ MB


In [136]:
alrernative_name.head()

,REGISTRY_ID,PGM_SYS_ACRNM,PGM_SYS_ID,ALTERNATIVE_NAME,ALTERNATIVE_NAME_TYPE
0,110000491735,NCDB,I10#1997082913928 1,MAPCO ALASKA PETROLEUM,PROGRAM NAME
1,110000491735,ICIS,2600029861,FLINT HILLS RESOURCES ALASKA LLC - PORT OF ANC...,PROGRAM NAME
2,110000491735,RCRAINFO,AKD000641852,WILLIAMS ALASKA PETROLEUM INC,PRIMARY
3,110000491735,TRIS,99501MPCLS1076O,FLINT HILLS RESOURCES ALASKA LLC ANCHORAGE TER...,PROGRAM NAME
4,110000491735,RCRAINFO,AKD980987499,OCEAN DOCK TERMINAL,PROGRAM NAME


In [129]:
environmental_interest = pd.read_csv("EPA API\\downloads\\national_combined\\NATIONAL_ENVIRONMENTAL_INTEREST_FILE.CSV")

In [130]:
environmental_interest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7104081 entries, 0 to 7104080
Data columns (total 14 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   REGISTRY_ID           int64 
 1   PGM_SYS_ACRNM         object
 2   PGM_SYS_ID            object
 3   INTEREST_TYPE         object
 4   FED_STATE_CODE        object
 5   START_DATE            object
 6   START_DATE_QUALIFIER  object
 7   END_DATE              object
 8   END_DATE_QUALIFIER    object
 9   SOURCE_OF_DATA        object
 10  LAST_REPORTED_DATE    object
 11  CREATE_DATE           object
 12  UPDATE_DATE           object
 13  ACTIVE_STATUS         object
dtypes: int64(1), object(13)
memory usage: 758.8+ MB


In [140]:
environmental_interest.head(3)

,REGISTRY_ID,PGM_SYS_ACRNM,PGM_SYS_ID,INTEREST_TYPE,FED_STATE_CODE,START_DATE,START_DATE_QUALIFIER,END_DATE,END_DATE_QUALIFIER,SOURCE_OF_DATA,LAST_REPORTED_DATE,CREATE_DATE,UPDATE_DATE,ACTIVE_STATUS
0,110000491735,OTAQREG,OTAQREG10028835,ETHANOL FACILITY/GASOLINE AND DIESEL PRODUCERS,FEDERAL,NaN,NaN,NaN,NaN,OTAQREG,NaN,11-JUN-18,NaN,A
1,110000491735,NPDES,AKR06AE27,STORM WATER INDUSTRIAL,FEDERAL,01-JUL-16,ORIGINAL PERMIT ISSUE DATE,NaN,NaN,ICIS,12-AUG-20,01-MAR-19,NaN,EFFECTIVE
2,110000491735,OTAQREG,OTAQREG10032817,ETHANOL FACILITY/GASOLINE AND DIESEL PRODUCERS,FEDERAL,NaN,NaN,NaN,NaN,OTAQREG,NaN,11-JUN-18,NaN,A


In [141]:
national_facility = pd.read_csv("EPA API\\downloads\\national_combined\\NATIONAL_FACILITY_FILE.CSV")

C:\Users\Smile2DeaTH\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (4,7,12,13,14,15,16,20,21,23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [142]:
national_facility.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4524900 entries, 0 to 4524899
Data columns (total 34 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   FRS_FACILITY_DETAIL_REPORT_URL  object 
 1   REGISTRY_ID                     int64  
 2   PRIMARY_NAME                    object 
 3   LOCATION_ADDRESS                object 
 4   SUPPLEMENTAL_LOCATION           object 
 5   CITY_NAME                       object 
 6   COUNTY_NAME                     object 
 7   FIPS_CODE                       object 
 8   STATE_CODE                      object 
 9   STATE_NAME                      object 
 10  COUNTRY_NAME                    object 
 11  POSTAL_CODE                     object 
 12  FEDERAL_FACILITY_CODE           object 
 13  FEDERAL_AGENCY_NAME             object 
 14  TRIBAL_LAND_CODE                object 
 15  TRIBAL_LAND_NAME                object 
 16  CONGRESSIONAL_DIST_NUM          object 
 17  CENSUS_BLOCK_CODE          

In [143]:
national_facility.head(1)

,FRS_FACILITY_DETAIL_REPORT_URL,REGISTRY_ID,PRIMARY_NAME,LOCATION_ADDRESS,SUPPLEMENTAL_LOCATION,CITY_NAME,COUNTY_NAME,FIPS_CODE,STATE_CODE,STATE_NAME,...,US_MEXICO_BORDER_IND,PGM_SYS_ACRNMS,LATITUDE83,LONGITUDE83,CONVEYOR,COLLECT_DESC,ACCURACY_VALUE,REF_POINT_DESC,HDATUM_DESC,SOURCE_DESC
0,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110000491735,OCEAN DOCK TERMINAL AND ANCHORAGE TERMINAL II,1076 OCEAN DOCK ROAD,NaN,ANCHORAGE,ANCHORAGE,02020,AK,ALASKA,...,NaN,"AIR:AK0000000202000032, AIRS/AFS:0202000032, B...",61.229579,-149.893094,EPA_SLT,INTERPOLATION-PHOTO,125.0,FACILITY CENTROID,NAD83,NaN


In [148]:
# percent of NA values 
round(sum(national_facility.LATITUDE83.isna())/ len(national_facility) *100,2)

29.84

# Using EPA REST API

# FEMA API

In [5]:
import json

In [6]:
f = open('FEMA API\\DisasterDeclarationsSummaries.json')
 
# returns JSON object as
# a dictionary
data = json.load(f)

# Iterating through the json
# list
data_list = []
for i in data['DisasterDeclarationsSummaries']:
    data_list.append(i)

data_2 = pd.DataFrame(data_list)

In [7]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   disasterNumber        1000 non-null   object
 1   ihProgramDeclared     1000 non-null   bool  
 2   iaProgramDeclared     1000 non-null   bool  
 3   paProgramDeclared     1000 non-null   bool  
 4   hmProgramDeclared     1000 non-null   bool  
 5   state                 1000 non-null   object
 6   declarationDate       1000 non-null   object
 7   fyDeclared            1000 non-null   int64 
 8   disasterType          1000 non-null   object
 9   incidentType          1000 non-null   object
 10  title                 1000 non-null   object
 11  incidentBeginDate     1000 non-null   object
 12  incidentEndDate       1000 non-null   object
 13  disasterCloseOutDate  1000 non-null   object
 14  declaredCountyArea    1000 non-null   object
 15  placeCode             1000 non-null   o

In [9]:
data_2['incidentType'].value_counts()

Flood               648
Tornado             165
Hurricane            84
Drought              54
Severe Storm(s)      13
Typhoon              10
Earthquake            9
Other                 8
Fire                  4
Toxic Substances      2
Volcano               1
Dam/Levee Break       1
Snow                  1
Name: incidentType, dtype: int64

In [297]:
with open('FEMA API\\DisasterDeclarationsSummaries.json', 'r') as openfile:
  
    # Reading from json file
    json_object = json.load(openfile)

In [430]:
#json_object

In [10]:
%pwd

'D:\\ALL_data_science\\Freelancing\\UPwork\\Super Software'

In [30]:
df = pd.read_csv('FEMA API/usfa_nfirs_2019_hazmat/USFA NFIRS 2019 Hazmat/NFIRS_FIRES_2019_011921/incidentaddress.txt', delimiter = "^",encoding='utf-8').strip()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xbf in position 14314: invalid start byte

In [28]:
list(df.columns)

['INCIDENT_KEY',
 'STATE',
 'FDID',
 'INC_DATE',
 'INC_NO',
 'EXP_NO',
 'AGENCY_NAM',
 'VERSION',
 'AG_ST_NUM',
 'AG_ST_PREF',
 'AG_STREET',
 'AG_ST_TYPE',
 'AG_ST_SUFF',
 'AG_APT_NO',
 'AG_CITY',
 'AG_STATE',
 'AG_ZIP5',
 'AG_ZIP4',
 'AG_PHONE',
 'AG_CASE_NO',
 'AG_ORI',
 'AG_FID',
 'AG_FDID']